In [1]:
from pymongo import MongoClient

mongo_client = MongoClient()
db = mongo_client['uvi_corpora']

In [2]:
from collections import OrderedDict

In [3]:
themroles = []
themrole_count_dict = OrderedDict()
for themrole_list in db.verbnet.find({}, {'themroles.themrole':1, '_id':0}):
    themroles.extend([themrole['themrole'] for themrole in themrole_list['themroles']])
# themroles_set = set(themroles)
for themrole in sorted(themroles):
    themrole_count_dict[themrole] = themrole_count_dict.get(themrole, 0) + 1


In [9]:
themroles_dict = OrderedDict()
import json
with open('themrole_defs.json') as td:
    themrole_defs = json.load(td)

for themrole_list in db.verbnet.find({}, {'themroles.themrole':1, 'class_id':1, '_id':0}):
    for tr in themrole_list['themroles']:
        if tr['themrole'] not in themroles_dict:
            themroles_dict[tr['themrole']] = {'count':1, 'vn_class_members':set([themrole_list['class_id']])}
            role_index = next((index for (index, d) in enumerate(themrole_defs) if d["name"] == "Patient"), None)
            themroles_dict[tr['themrole']]['description'] = themrole_defs[role_index]['description']
            themroles_dict[tr['themrole']]['example'] = themrole_defs[role_index]['example']            
        else:
            themroles_dict[tr['themrole']]['count']+=1
            themroles_dict[tr['themrole']]['vn_class_members'].add(themrole_list['class_id'])
sorted(themroles_dict.keys())

['Affector',
 'Agent',
 'Asset',
 'Attribute',
 'Beneficiary',
 'Causer',
 'Co-Agent',
 'Co-Patient',
 'Co-Theme',
 'Context',
 'Destination',
 'Duration',
 'Experiencer',
 'Extent',
 'Final_Time',
 'Goal',
 'Initial_Location',
 'Initial_State',
 'Instrument',
 'Location',
 'Manner',
 'Material',
 'Path',
 'Patient',
 'Pivot',
 'Precondition',
 'Predicate',
 'Product',
 'Recipient',
 'Reflexive',
 'Result',
 'Source',
 'Stimulus',
 'Theme',
 'Time',
 'Topic',
 'Trajectory',
 'Value']

In [5]:
#Can't simply match on cases where vs_features is not None, so instead we match cases where the datatype is a list (mongodb array) 
vs_features = []
vs_features_count_dict = OrderedDict()
for doc in list(db.verbnet.find({'members.vs_features': {'$type': 'array'}}, {'members.vs_features':1, 'class_id':1, '_id':0})):
    for vs_features_dict in doc['members']:
        if vs_features_dict['vs_features'] is not None:
            for vs in vs_features_dict['vs_features']:
                if vs not in vs_features_count_dict:
                    vs_features_count_dict[vs]={'count':1,
                                                'vn_class_mem':set([doc['class_id']])}
                else:
                    vs_features_count_dict[vs]['count'] += 1
                    vs_features_count_dict[vs]['vn_class_mem'].add(doc['class_id'])
print(vs_features_count_dict)


OrderedDict([('+speed', {'count': 62, 'vn_class_mem': {'run-51.3.2', 'run-51.3.2-2', 'run-51.3.2-1', 'run-51.3.2-2-1'}}), ('+bodily_manner', {'count': 62, 'vn_class_mem': {'run-51.3.2', 'run-51.3.2-2', 'run-51.3.2-1', 'run-51.3.2-2-1'}}), ('+orientation', {'count': 17, 'vn_class_mem': {'run-51.3.2', 'run-51.3.2-2', 'run-51.3.2-1', 'run-51.3.2-2-1'}}), ('+attitude', {'count': 26, 'vn_class_mem': {'run-51.3.2', 'run-51.3.2-2', 'run-51.3.2-1'}}), ('+path_shape', {'count': 22, 'vn_class_mem': {'run-51.3.2', 'run-51.3.2-2', 'run-51.3.2-1'}}), ('+purpose', {'count': 20, 'vn_class_mem': {'run-51.3.2', 'run-51.3.2-2', 'run-51.3.2-1', 'run-51.3.2-2-1'}})])


In [12]:
predicates = []
predicate_count_dict = OrderedDict()
for doc in list(db.verbnet.find({}, {'frames.semantics.predicate':1, '_id':0})):
    for frame in doc['frames']:
        predicates_list = [predicate['predicate'] for predicate in frame['semantics']]
        predicates.extend(predicates_list)

for predicate in sorted(predicates):
    predicate_count_dict[predicate] = predicate_count_dict.get(predicate, 0) + 1

set(predicates)

{'Adv',
 'about',
 'act',
 'adjust',
 'admit',
 'adopt',
 'agree',
 'alive',
 'allow',
 'apart',
 'appear',
 'apply_heat',
 'apply_material',
 'approve',
 'assess',
 'attached',
 'attempt',
 'attract',
 'authority_relationship',
 'avoid',
 'base',
 'begin',
 'believe',
 'benefit',
 'body_motion',
 'body_process',
 'body_reflex',
 'calculate',
 'capacity',
 'cause',
 'change_value',
 'characterize',
 'charge',
 'conclude',
 'confined',
 'conflict',
 'confront',
 'consider',
 'conspire',
 'contact',
 'contain',
 'continue',
 'convert',
 'cooked',
 'cooperate',
 'cope',
 'correlate',
 'cost',
 'covered',
 'created_image',
 'declare',
 'dedicate',
 'defend',
 'degradation_material_integrity',
 'delay',
 'depend',
 'describe',
 'designated',
 'desire',
 'destroyed',
 'different',
 'direction',
 'disappear',
 'discomfort',
 'discover',
 'do',
 'earn',
 'emit',
 'emotional_state',
 'end',
 'enforce',
 'ensure',
 'equals',
 'exceed',
 'exert_force',
 'exist',
 'experience',
 'express',
 'fille

In [ ]:
predicates_dict = OrderedDict()
for doc in list(db.verbnet.find({}, {'frames.semantics.predicate':1, 'class_id':1, '_id':0})):
    for frame in doc['frames']:
        for pr in frame['semantics']:
            if pr['predicate'] not in predicates_dict:
                predicates_dict[pr['predicate']] = {'count':1, 'vn_class_members':set([doc['class_id']])}
            else:
                predicates_dict[pr['predicate']]['count']+=1
                predicates_dict[pr['predicate']]['vn_class_members'].add(doc['class_id'])

In [ ]:
list(db.verbnet.find({},{'frames.semantics.predicate':1, '_id':0}))

In [ ]:
predicate_count_dict = OrderedDict()
for doc in list(db.verbnet.find({}, {'frames.description':1, '_id':0})):
    for frame in doc['frames']:
        print(frame['description']['primary'])
        #         primary = frame['primary']

In [ ]:
# Get all frame types by querying db
frame_types = []
frame_type_count_dict = OrderedDict()
for doc in list(db.verbnet.find({}, {'frames.description':1, '_id':0})):
    for frame in doc['frames']:
        frame_types_list = frame['description']['primary']
        frame_types.append(frame_types_list)
for f_type in sorted(frame_types):
    frame_type_count_dict[f_type] = frame_type_count_dict.get(f_type, 0) + 1

In [ ]:
np_themroles = {}
# np_themrole_count_dict = OrderedDict()
for frame in db.verbnet.find({}, {'frames.syntax':1, 'class_id': 1, '_id':0}):
    for syn_list in frame['frames']:
        for arg in syn_list['syntax']:
            if arg['arg_type']=='NP':
                if "?" in arg['value']:
                    print(arg['value'], syn_list['syntax'])
                if arg['value'] not in np_themroles:
                    ## Some roles have a ? mark at the start. Replace them with a "".
                    np_themroles[arg['value'].replace("?", "")] = {'count':1, 'vn_class_members':set([frame['class_id']])}
                else:
                    np_themroles[arg['value'].replace("?", "")]['count'] += 1
                    np_themroles[arg['value'].replace("?", "")]['vn_class_members'].add(frame['class_id'].replace("?", ""))

In [ ]:
### Find selectional restrictions and add member vn classes to respective restrictions as the main dict elements so that its easy for html to parse
from pprint import pprint as pp
sr_dict = {}
for vn_class in list(db.verbnet.find({'frames.syntax.synrestrs.synrestr_list.type': {'$exists':1}}, {'frames.syntax.synrestrs.synrestr_list':1, 'class_id':1, '_id':0})):
    for syn in vn_class['frames']:
        for el in syn['syntax']:
            if el['synrestrs']['synrestr_list']:
                for sr_el in el['synrestrs']['synrestr_list']:
                    if sr_el['value']=='-':
                        print(sr_el['type'])
                    if sr_el['type'] not in sr_dict:
                        sr_dict[sr_el['type']] = {'Frames':{sr_el['value']:{'count':1,
                                                                            'vn_mem':set([vn_class['class_id']])}}}
                    else:
                        if sr_el['value'] not in sr_dict[sr_el['type']]['Frames']:
                            sr_dict[sr_el['type']]['Frames'][sr_el['value']] = {'count':1,
                                                                                 'vn_mem': set([vn_class['class_id']])}                          
                        else:
                            sr_dict[sr_el['type']]['Frames'][sr_el['value']]['count'] +=1
                            sr_dict[sr_el['type']]['Frames'][sr_el['value']]['vn_mem'].add(vn_class['class_id'])
for vn_class in list(db.verbnet.find({}, {'themroles.selrestrs.selrestrs_list':1, 'class_id':1, '_id':0})):
    for selres in vn_class['themroles']:
        if isinstance(selres['selrestrs']['selrestrs_list'], list):
            for sel_elem in selres['selrestrs']['selrestrs_list']:
                if sel_elem['type']:
                    if sel_elem['type'] not in sr_dict:
                        sr_dict[sel_elem['type']] = {'Themroles': {sel_elem['value']:{'count':1, 
                                                                'vn_mem': set([vn_class['class_id']])}}}
                    elif 'Themroles' not in sr_dict[sel_elem['type']]:
                        sr_dict[sel_elem['type']]['Themroles'] = {sel_elem['value']:{'count':1,
                                                                                     'vn_mem': set([vn_class['class_id']])}}
                    else:
                        if sel_elem['value'] not in sr_dict[sel_elem['type']]['Themroles']:
                            sr_dict[sel_elem['type']]['Themroles'][sel_elem['value']] = {'count':1,'vn_mem': set([vn_class['class_id']])}
                        else:
                            sr_dict[sel_elem['type']]['Themroles'][sel_elem['value']]['count'] +=1
                            sr_dict[sel_elem['type']]['Themroles'][sel_elem['value']]['vn_mem'].add(vn_class['class_id'])
                        